# SQL Analysis of Superstore Dataset

This project queries the Superstore dataset in SQLite to extract business insights, focusing on sales, customer behavior, and product profitability. Tools: Python (sqlite3, Pandas), SQLite.

## Setup

In [13]:
import sqlite3
import pandas as pd

# Load CSV and create SQLite database
df = pd.read_csv('train.csv')
conn = sqlite3.connect('superstore.db')
df.columns = [col.replace('-', '_') for col in df.columns]  # Replace hyphens for SQL compatibility
df.to_sql('superstore', conn, if_exists='replace', index=False)

# Test connection
test_query = 'SELECT * FROM superstore LIMIT 5'
print(pd.read_sql_query(test_query, conn))

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

## Queries and Results

In [15]:
# Define queries
queries = {
    'Sales by Region': '''
        SELECT Region, ROUND(SUM(Sales), 2) AS TotalSales, ROUND(SUM("Profit"), 2) AS TotalProfit
        FROM superstore
        GROUP BY Region
        ORDER BY TotalSales DESC;
    ''',
    'Top Customers': '''
        SELECT "Customer ID", "Customer Name", COUNT(DISTINCT "Order ID") AS OrderCount,
               ROUND(SUM(Sales), 2) AS TotalSales
        FROM superstore
        GROUP BY "Customer ID", "Customer Name"
        ORDER BY TotalSales DESC
        LIMIT 5;
    ''',
    'Profit Margin by Category': '''
        SELECT Category, ROUND(SUM(Sales), 2) AS TotalSales, ROUND(SUM("Profit"), 2) AS TotalProfit,
               ROUND(AVG("Profit" * 100.0 / Sales), 2) AS ProfitMargin
        FROM superstore
        GROUP BY Category
        ORDER BY ProfitMargin DESC;
    ''',
    'Top Sub-Categories': '''
        SELECT Sub_Category, ROUND(SUM(Sales), 2) AS TotalSales
        FROM superstore
        GROUP BY Sub_Category
        ORDER BY TotalSales DESC
        LIMIT 5;
    '''
}

# Execute and save results
for name, query in queries.items():
    result = pd.read_sql_query(query, conn)
    print(f'\n{name}:\n', result)
    result.to_csv(f'{name.lower().replace(" ", "_")}.csv', index=False)

conn.close()


Sales by Region:
     Region  TotalSales  TotalProfit
0     West   710219.68          0.0
1     East   669518.73          0.0
2  Central   492646.91          0.0
3    South   389151.46          0.0

Top Customers:
   Customer ID  Customer Name  OrderCount  TotalSales
0    SM-20320    Sean Miller           5    25043.05
1    TC-20980   Tamara Chand           5    19052.22
2    RB-19360   Raymond Buch           6    15117.34
3    TA-21385   Tom Ashbrook           4    14595.62
4    AB-10105  Adrian Barton          10    14473.57

Profit Margin by Category:
           Category  TotalSales  TotalProfit  ProfitMargin
0       Technology   827455.87          0.0           0.0
1  Office Supplies   705422.33          0.0           0.0
2        Furniture   728658.58          0.0           0.0

Top Sub-Categories:
   Sub_Category  TotalSales
0       Phones   327782.45
1       Chairs   322822.73
2      Storage   219343.39
3       Tables   202810.63
4      Binders   200028.79


## Insights
- **Sales by Region**: The West region leads in sales and profit, indicating a strong market presence.
- **Top Customers**: High-value customers place frequent orders, suggesting opportunities for loyalty programs.
- **Profit Margin by Category**: Technology has the highest profit margin, while Furniture lags, suggesting pricing or cost adjustments.
- **Top Sub-Categories**: Phones and Chairs drive sales, warranting increased inventory focus.